In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/data-engineering-zoomcamp/01-docker-terraform/terraform/keys/my-creds.json"
os.environ["BUCKET_URL"] = "gs://nth-weft-417606-ny-taxi-bucket"

In [5]:
# Install for production
!uv pip install dlt[bigquery,gs]


Resolved 75 packages in 1.33s                                        
Prepared 54 packages in 643ms                                            
Uninstalled 1 package in 96ms
░░░░░░░░░░░░░░░░░░░░ [0/57] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 57 packages in 12.36s                             
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.3
 + aiosignal==1.4.0
 + click==8.3.1
 + cryptography==46.0.4
 + db-dtypes==1.5.0
 + dlt==1.21.0
 + frozenlist==1.8.0
 + fsspec==2026.2.0
 + gcsfs==2026.2.0
 + gitdb==4.0.12
 + gitpython==3.1.46
 + giturlparse==0.14.0
 + google-api-core==2.29.0
 + google-auth==2.48.0
 + google-auth-oauthlib==1.2.4
 + google-cloud-

In [6]:
import dlt
import requests
import pandas as pd
from dlt.destinations import filesystem
from io import BytesIO

In [7]:
# Define a dlt source to download and process Parquet files as resources
@dlt.source(name="rides")
def download_parquet():
    prefix = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"
    for month in range(1, 7):
        file_name = f"yellow_tripdata_2024-0{month}.parquet"
        url = f"{prefix}_2024-0{month}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))

        # Return the dataframe as a dlt resource for ingestion
        yield dlt.resource(df, name=file_name)


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination=filesystem(layout="{schema_name}/{table_name}.{ext}"),
    dataset_name="rides_dataset",
)

# Run the pipeline to load Parquet data into DuckDB
load_info = pipeline.run(download_parquet(), loader_file_format="parquet")

# Print the results
print(load_info)

Pipeline rides_pipeline load step completed in 22.03 seconds
1 load package(s) were loaded to destination filesystem and into dataset rides_dataset
The filesystem destination used gs://nth-weft-417606-ny-taxi-bucket location to store data
Load package 1770550868.9556327 is LOADED and contains no failed jobs
